In [1]:
####################################################################################
#
# Preprocessing_Patchfiles_and_Initial_Migrationfile.ipynb - script for pre-processing patchfiles and initial database migration file for the Weberp project.
# Copyright (C) 2023  Sravani Namburi
#
# Preprocessing_Patchfiles_and_Initial_Migrationfile.ipynb program is free software; you can redistribute it and/or modify
# it under the terms of the GNU General Public License as published by
# the Free Software Foundation; either version 3 of the License, or
# (at your option) any later version.
# 
# Preprocessing_Patchfiles_and_Initial_Migrationfile.ipynb program is distributed in the hope that it will be useful,
# but WITHOUT ANY WARRANTY; without even the implied warranty of
# MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
# GNU General Public License for more details.
# 
# You should have received a copy of the GNU General Public License along
# with Preprocessing_Patchfiles_and_Initial_Migrationfile.ipynb program; If not, see <https://www.gnu.org/licenses/>.
#
####################################################################################

In [2]:
import os
import re
import io
from pathlib import Path
import os.path
import networkx as nx

## Pre-Processing Patch Files

In [3]:
# duplicate statements with minor changes and directly remove the second occurrence 
lines_to_skip = [
    "ALTER TABLE suppliers ADD COLUMN `factorcompanyid` int(11) NOT NULL DEFAULT '1';",
    "ALTER TABLE bankaccounts ADD COLUMN `currcode` char(3) NOT NULL;",
    "ALTER TABLE  `stockmoves` ADD  `userid` VARCHAR( 20 ) CHARACTER SET utf8 COLLATE utf8_general_ci NOT NULL AFTER  `trandate`;",
    "ALTER TABLE `custcontacts` ADD `statement` TINYINT(4) NOT NULL DEFAULT 0;",
    "ALTER TABLE `assetmanager` CHANGE `datepurchased` `datepurchased` DATE NOT NULL DEFAULT '1000-01-01';",
    "ALTER TABLE `custbranch` DROP COLUMN `vtiger_accountid`;",
    "ALTER TABLE `salesorders` DROP COLUMN `vtiger_accountid`;",
    "ALTER TABLE `stockmaster` DROP COLUMN `vtiger_productid`;",
    "ALTER TABLE `custbranch` CHANGE `area` `area` CHAR( 3 ) NOT NULL;",
    "ALTER TABLE `salesman` CHANGE `salesmancode` `salesmancode` VARCHAR( 4 ) NOT NULL DEFAULT '';",
    "ALTER TABLE `salesorders` CHANGE `orderno` `orderno` INT( 11 ) NOT NULL;",
    "ALTER TABLE `workorders` ADD CONSTRAINT `worksorders_ibfk_1` FOREIGN KEY (`loccode`) REFERENCES `locations` (`loccode`);",
    "ALTER TABLE `www_users` DROP `pinno` ;",
    "ALTER TABLE `www_users` DROP `swipecard` ;",
    "ALTER TABLE `bankaccounts` ADD INDEX ( `currcode` ) ;",
    
    #duplicate same statements
    "ALTER table stockmoves CHANGE reference reference varchar(100) NOT NULL DEFAULT '';",
    "ALTER TABLE `chartmaster` ADD `cashflowsactivity` TINYINT(1) NOT NULL DEFAULT '-1' COMMENT 'Cash flows activity' AFTER `group_`"
    
]

def remove_comments(sql):
    return re.sub(r'--.*?\n|/\*.*?\*/', '', sql, flags=re.DOTALL)

def remove_convert_statements(sql):
    convert_pattern = r'ALTER\s+TABLE\s+stockcatproperties\s+CONVERT\s+TO\s+CHARACTER\s+SET\s+utf8;'
    return re.sub(f'(?!{convert_pattern})ALTER\s+TABLE.*?CONVERT.*?CHARACTER\s+SET.*?;', '', sql, flags=re.IGNORECASE)



def remove_fk(sql):
    sql = re.sub(r',?\s*CONSTRAINT\s+FOREIGN\s+KEY\s+\(`?(\w+)`?\)\s+REFERENCES\s+`?\w+`?\s+\(`?\w+`?\)\s*(?:,\s*INDEX\s+\(`?\w+`?\)`?)?\s*(?=,|\)|;)', '', sql, flags=re.IGNORECASE)
    sql = re.sub(r'\s*CONSTRAINT\s+FOREIGN\s+KEY\s+\(`?(\w+)`?\)\s+REFERENCES\s+`?\w+`?\s+\(`?\w+`?\)\s*(?:,\s*INDEX\s+\(`?\w+`?\)`?)?\s*(?=,|\)|;)', '', sql, flags=re.IGNORECASE)
    sql = re.sub(r'CONSTRAINT\s+FOREIGN\s+KEY\s+\(`?userid`?\)\s+REFERENCES\s+`?www_users`?\s*\(`?userid`?\)\s*(?=,|\)|;)', '', sql, flags=re.IGNORECASE)
    sql = re.sub(r',?\s*CONSTRAINT\s+`?\s*custitem\s+_ibfk_1`?\s+FOREIGN\s+KEY\s*\(.*?\)\s+REFERENCES\s+`?stockmaster`?\s*\(.*?\)', '', sql, flags=re.IGNORECASE)
    sql = re.sub(r',?\s*CONSTRAINT\s+`?\s*custitem\s+_ibfk_2`?\s+FOREIGN\s+KEY\s*\(.*?\)\s+REFERENCES\s+`?debtorsmaster`?\s*\(.*?\)', '', sql, flags=re.IGNORECASE) 
    sql = re.sub(r'\s*constraint\s+foreign\s+key\s*\(.*?\)\s+REFERENCES\s+`?stockmaster`?\s*\(.*?\)\s*,?', '', sql, flags=re.IGNORECASE)
    sql = re.sub(r',?\s*CONSTRAINT\s+FOREIGN\s+KEY\s+\(`?stockid`?,\s*`?serialno`?\)\s+REFERENCES\s+`?stockserialitems`?\(`?stockid`?,\s*`?serialno`?\)\s*,?', '', sql, flags=re.IGNORECASE)
    sql = re.sub(r',\s*(?=,|\)|;)', '', sql)
    return sql


def remove_specific_fk(sql):
    return re.sub(r'\s*CONSTRAINT\s+FOREIGN\s+KEY\s*\(.*?\)\s+REFERENCES\s+`?locations`?\s*\(.*?\)\s*,?', '', sql, flags=re.IGNORECASE)

    
def replace_date_and_default_values(sql):
    sql = sql_content.replace('0000-00-00', '2023-01-01')
    sql = sql_content.replace('0000-00-00 00:00:00', '2023-01-01 00:00:00')

    # Remove default values for specific data types
    sql = re.sub(r'(TEXT|MEDIUMTEXT|BLOB|GEOMETRY|JSON|text|mediumtext|blob|geometry|json) .*? DEFAULT \'\',', r'\1 NOT NULL,', sql_content)
    return sql

def extract_statements(sql):
    statements = re.findall(r'(ALTER TABLE.*?;|CREATE TABLE.*?;|DROP TABLE.*?;|RENAME TABLE.*?;|CREATE table.*?;|ALTER table.*?;)', sql, flags=re.DOTALL)
    for statement in statements:
        if 'ALTER TABLE' in statement or 'ALTER table' in statement:
            if 'ADD FOREIGN KEY' in statement or 'DROP FOREIGN KEY' in statement or 'ADD KEY'in statement or\
            'DROP foreign key' in statement or 'DROP KEY' in statement or 'ADD CONSTRAINT' in statement or 'ADD `cashflowsactivity`' in statement:
                continue   
                
        elif 'CREATE TABLE' in statement:
            statement = re.sub(r',?\s*CONSTRAINT\s+`?\w+`?\s+FOREIGN\s+KEY\s*\(.*?\)\s+REFERENCES\s+`?\w+`?\s*\(.*?\)', '', statement, flags=re.IGNORECASE)
       
        elif 'DROP' in statement:
            if 'DROP KEY' in statement:
                continue    
                
        yield statement


cwd = os.getcwd()
input_folder = os.path.join(cwd, 'Patchfiles')
output_folder = os.path.join(cwd, 'Patchfiles_Mod')

if not os.path.exists(output_folder):
    os.makedirs(output_folder)

for filename in os.listdir(input_folder):

    if filename.endswith('.sql'):
        with open(os.path.join(input_folder, filename), 'r') as input_file:
            sql_content = input_file.read()

        sql_content = remove_comments(sql_content)
        sql_content = remove_convert_statements(sql_content)
        sql_content = replace_date_and_default_values(sql_content)
        sql_content = remove_fk(sql_content)
        sql_content = remove_specific_fk(sql_content)
  
     
        statements = extract_statements(sql_content)

        output_file = os.path.join(output_folder, filename.replace('.sql', '_mod.sql'))

        with open(output_file, 'w') as output_file:
   
            for statement in statements:

                if statement.strip() not in lines_to_skip:
                    
                    output_file.write(statement.strip() + '\n')


In [4]:
# write statements in files, these are skiped above code because duplicate but we need first statements , that is way adding in first occurence. 

cwd = os.getcwd()
input_folder = output_folder = os.path.join(cwd, 'Patchfiles_Mod')

file_path = os.path.join(input_folder, 'upgrade4.11.2-4.11.3_mod.sql')
with open(file_path, 'a') as file:
    file.write("ALTER table stockmoves CHANGE reference reference varchar(100) NOT NULL DEFAULT '';\n")

file_path = os.path.join(input_folder, 'upgrade4.13-4.13.1_mod.sql')
with open(file_path, 'a') as file:
    file.write("ALTER TABLE `chartmaster` ADD `cashflowsactivity` TINYINT(1) NOT NULL DEFAULT '-1' COMMENT 'Cash flows activity' AFTER `group_`;\n")

## Pre-processing Initial Database Migration File

In [5]:
#!/usr/bin/env python

def remove_sngl_comments(Text):
    Comments_lst = re.findall(r"--.*|#.*", Text)
    Comments_lst.sort(reverse=True)
    for Comment in Comments_lst:
        Text = Text.replace(Comment, "")
    return Text.strip()

def remove_mult_comments(Text):
    idx = Text.find("/*")
    if idx != -1:
        return Text[:idx]
    return Text

def mod_query(Query):
    New_Query = ""
    Lines = Query.split("\n")
    for Line in Lines:
        Line = Line.replace("\t", " ")
        Line = Line.replace("IF NOT EXISTS", "")
        Line = Line.replace("`href` varchar(200) NOT NULL DEFAULT '","`href` varchar(200) NOT NULL DEFAULT '#',")
        Line = Line.replace("TYPE=InnoDB", "ENGINE=InnoDB")
        Line = Line.replace("TYPE=MyISAM", "ENGINE=MyISAM")
        Line = Line.replace("NULLLL", "NULL")
        Line = Line.replace("''''", "")
        New_Query = "{} {}".format(New_Query, Line)

    New_Query = " ".join(New_Query.split())
    New_Query = New_Query.replace(", )", ")").replace("( ", "(").replace(" )", ")")
    return New_Query

def main():
    cwd = os.getcwd()
    parent_dir = os.path.dirname(cwd)
    sql_file_path = os.path.join(parent_dir, 'weberp-demo-original.sql')

    with io.open(sql_file_path, mode="r", encoding="utf-8") as f:
        Text = f.read()
        Queries = Text.split(";")

        Lines_mod = list()
        Lines = Text.split("\n")
        for Line in Lines:
            Line_mod = remove_sngl_comments(Line)
            Line_mod = remove_mult_comments(Line_mod)
            Lines_mod.append(Line_mod)

        Text = "\n".join(Lines_mod)
        Queries = Text.split(";")
        Queries_list = list()
        for Query in Queries:
            if "create table" in Query.lower():
                Query_mod = mod_query(Query)
                Queries_list.append(Query_mod)


    Filename_mod = "{}/weberp-demo-original_mod.sql".format(parent_dir)
    with io.open(Filename_mod, mode="w", encoding="utf-8") as File_mod:
        for Query in Queries_list:
            File_mod.write("{};\n\n".format(Query))
        
    print("Extraction and transformation of Initial database schema file completed!")

if __name__ == "__main__":
    main()


Extraction and transformation of Initial database schema file completed!


### Topological Sort

In [6]:
# Extract the table names and their dependencies from the SQL file.
cwd = os.getcwd()
parent_dir = os.path.dirname(cwd)
Initial_Migration_File_Path = os.path.join(parent_dir, 'weberp-demo-original_mod.sql') 

Create_table = re.compile(r'CREATE TABLE ([^\s\(]+)')
References_pattern = re.compile(r'REFERENCES ([^\s\(]+)')

def extract_dependencies(content):
    """
    Extract dependencies from the SQL statements.
    """
    Dependencies = []
    References = re.findall(References_pattern, content)
    for reference in References:
        if reference not in Dependencies:
            Dependencies.append(reference)
    return Dependencies

Tables_and_dependencies_list = []

with open(Initial_Migration_File_Path, 'r') as file:
    Content = file.read()

Statements = Content.split(';')
for statement in Statements:
    Table_name = re.findall(Create_table, statement)
    if Table_name:
        Table_name = Table_name[0]
        Dependencies = extract_dependencies(statement)
        Tables_and_dependencies_list.append((Table_name, Dependencies))

for table, dependency in Tables_and_dependencies_list:
    print(f'Table: {table}')
    print(f'Dependencies: {dependency}')

Table: `accountgroups`
Dependencies: []
Table: `accountsection`
Dependencies: []
Table: `areas`
Dependencies: []
Table: `bankaccounts`
Dependencies: ['`chartmaster`']
Table: `banktrans`
Dependencies: ['`systypes`', '`bankaccounts`']
Table: `bom`
Dependencies: ['`stockmaster`', '`workcentres`', '`locations`']
Table: `buckets`
Dependencies: ['`workcentres`']
Table: `chartdetails`
Dependencies: ['`chartmaster`', '`periods`']
Table: `chartmaster`
Dependencies: ['`accountgroups`']
Table: `cogsglpostings`
Dependencies: []
Table: `companies`
Dependencies: []
Table: `config`
Dependencies: []
Table: `contractbom`
Dependencies: ['`workcentres`', '`locations`', '`stockmaster`']
Table: `contractreqts`
Dependencies: ['`contracts`']
Table: `contracts`
Dependencies: ['`custbranch`', '`stockcategory`', '`salestypes`']
Table: `currencies`
Dependencies: []
Table: `custallocns`
Dependencies: ['`debtortrans`']
Table: `custbranch`
Dependencies: ['`debtorsmaster`', '`areas`', '`salesman`', '`locations`', '`

In [7]:
# count of dependency tables

cwd = os.getcwd()
parent_dir = os.path.dirname(cwd)
Initial_Migration_File_Path = os.path.join(parent_dir, 'weberp-demo-original_mod.sql') 


Tables_with_Dependencies = set()
Create_Table_Pattern = re.compile(r'CREATE TABLE ([^\s\(]+)')
References_Pattern = re.compile(r'REFERENCES ([^\s\(]+)')

def extract_dependencies(content):
    """
    Extract dependencies from the SQL statements.
    """
    references = re.findall(References_Pattern, content)
    for reference in references:
        Tables_with_Dependencies.add(reference)


with open(Initial_Migration_File_Path, 'r') as file:
    content = file.read()


statements = content.split(';')
for statement in statements:
    table_name = re.findall(Create_Table_Pattern, statement)
    if table_name:
        table_name = table_name[0]
        extract_dependencies(statement)

Total_Dependency_Table_Count = len(Tables_with_Dependencies)
print(f'Total Count of Dependency Tables: {Total_Dependency_Table_Count}')
print(Tables_with_Dependencies)

Total Count of Dependency Tables: 33
{'`taxauthorities`', '`debtortrans`', '`currencies`', '`areas`', '`purchorders`', '`contracts`', '`shipments`', '`chartmaster`', '`shippers`', '`salescat`', '`salesorders`', '`holdreasons`', '`salesman`', '`reportheaders`', '`workcentres`', '`locations`', '`stockserialitems`', '`suppliers`', '`debtorsmaster`', '`purchorderdetails`', '`salestypes`', '`securitytokens`', '`stockcategory`', '`securityroles`', '`stockmaster`', '`supptrans`', '`stockmoves`', '`paymentterms`', '`systypes`', '`custbranch`', '`periods`', '`accountgroups`', '`bankaccounts`'}


In [8]:
# Represent the tables and their dependencies as a directed graph.

Graph = nx.DiGraph()
for Table, _ in Tables_and_dependencies_list:
    Graph.add_node(Table)

for Table, Dependencies in Tables_and_dependencies_list:
    for dependency in Dependencies:
        Graph.add_edge(dependency, Table)

Sorted_Tables = list(nx.topological_sort(Graph))
print("Count of sorted tables:", len(Sorted_Tables))
for Table in Sorted_Tables:
    print(Table)

Count of sorted tables: 78
`accountgroups`
`accountsection`
`areas`
`cogsglpostings`
`companies`
`config`
`currencies`
`edi_orders_seg_groups`
`edi_orders_segs`
`ediitemmapping`
`edimessageformat`
`holdreasons`
`lastcostrollup`
`locations`
`paymentmethods`
`paymentterms`
`periods`
`reportheaders`
`salescat`
`salesglpostings`
`salesman`
`salestypes`
`scripts`
`securityroles`
`securitytokens`
`shippers`
`stockcategory`
`systypes`
`unitsofmeasure`
`chartmaster`
`workcentres`
`www_users`
`salesanalysis`
`reportcolumns`
`debtorsmaster`
`discountmatrix`
`securitygroups`
`freightcosts`
`recurringsalesorders`
`stockmaster`
`bankaccounts`
`chartdetails`
`gltrans`
`taxauthorities`
`buckets`
`bom`
`contractbom`
`locstock`
`loctransfers`
`prices`
`recurrsalesorderdetails`
`salescatprod`
`stockcheckfreeze`
`stockcounts`
`stockmoves`
`stockserialitems`
`worksorders`
`banktrans`
`custbranch`
`suppliers`
`taxauthlevels`
`stockserialmoves`
`contracts`
`debtortrans`
`salesorders`
`purchdata`
`purchorder

In [9]:
# Perform topological sorting on the graph to obtain the correct order of tables.Output the tables in the sorted order.
#if already files in folder, remove and run this code.

cwd = os.getcwd()
parent_dir = os.path.dirname(cwd)

Old_Filename = "weberp-demo-original_mod.sql"
New_Filename = "weberp-demo-original_mod-sort.sql"

Old_FilePath = os.path.join(parent_dir, Old_Filename)
New_FilePath = os.path.join(parent_dir, New_Filename)

if os.path.exists(Old_FilePath):
    with open(Old_FilePath, 'r') as file:
        Content = file.read()

        Mod_Content = ""
        for Table in Sorted_Tables:
            Table_Creation_Statement = re.search(r'CREATE TABLE {}(.+?);'.format(Table), Content, re.DOTALL)
            if Table_Creation_Statement:
                Mod_Content += Table_Creation_Statement.group(0) + "\n"
                
        with open(New_FilePath, 'w') as file:
            file.write(Mod_Content)

    with open(New_FilePath, 'w') as file:
        file.write(Mod_Content)

    os.remove(Old_FilePath)   

### Remove foreign key constraints

In [10]:
# remove foreign key constraints in create table.


def remove_foreign_key(sql):
    return re.sub(r',?\s*CONSTRAINT\s+`?\w+`?\s+FOREIGN\s+KEY\s*\(.*?\)\s+REFERENCES\s+`?\w+`?\s*\(.*?\)', '', sql, flags=re.IGNORECASE)


cwd = os.getcwd()
parent_dir = os.path.dirname(cwd)

input_file = os.path.join(parent_dir, 'weberp-demo-original_mod-sort.sql')
output_file = os.path.join(parent_dir, 'weberp-demo-original_mod-sort-fk.sql')

if os.path.exists(input_file):
    with open(input_file, 'r') as file:
        sql_content = file.read()

    
    statements = sql_content.split(';')

    with open(output_file, 'w') as file:
        for statement in statements:
            statement = statement.strip()

            if 'CREATE TABLE' in statement:
                clean_create_table_statement = remove_foreign_key(statement)
                file.write(clean_create_table_statement + ';\n')
            elif statement.strip():
                file.write(statement + ';\n')
else:
    print("Input file 'weberp-demo_mod.sql' not found.")


### Remove default values for specific data types

In [11]:
#Extract create table statements from initial database migration files and remove default values for selected datatypes

def extract_create_table_statements(sql_content):
    create_table_pattern = re.compile(r'CREATE TABLE .*?;', re.DOTALL)
    create_table_statements = create_table_pattern.findall(sql_content)
    return create_table_statements


def replace_date_and_default_values(sql_content):
    # Replace date values
    sql_content = sql_content.replace('0000-00-00', '2023-01-01')
    sql_content = sql_content.replace('0000-00-00 00:00:00', '2023-01-01 00:00:00')
    
    # Remove default values for specific data types
    sql_content = re.sub(r'(TEXT|MEDIUMTEXT|BLOB|GEOMETRY|JSON|text|mediumtext|blob|geometry|json) .*? DEFAULT \'\',', r'\1 NOT NULL,', sql_content)
    
    return sql_content


cwd = os.getcwd()
parent_dir = os.path.dirname(cwd)

sql_file_path = os.path.join(parent_dir, 'weberp-demo-original_mod-sort-fk.sql')

with open(sql_file_path, 'r') as file:
    sql_file_content = file.read()

create_table_statements = extract_create_table_statements(sql_file_content)
modified_create_table_statements = [replace_date_and_default_values(statement) for statement in create_table_statements]

output_file_path = os.path.join(parent_dir, 'weberp-demo-initial-migrationfile.sql')
with open(output_file_path, 'w') as file:
    file.write('\n'.join(modified_create_table_statements))